In [1]:
import numpy as np
import onnxruntime as ort
from pathlib import Path
TOKENIZER_PATH = 'encoder.onnx'
DETOKENIZER_PATH = 'decoder.onnx'
INPUT_SHAPE = (256, 160)
options = ort.SessionOptions()
tokenizer_session = ort.InferenceSession(TOKENIZER_PATH, options, ['CUDAExecutionProvider'])
decoder_session = ort.InferenceSession(DETOKENIZER_PATH, options, ['CUDAExecutionProvider'])

2024-07-17 17:35:36.824266537 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 2 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2024-07-17 17:35:36.826129599 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-07-17 17:35:36.826138937 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2024-07-17 17:35:37.430621055 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have 

In [2]:
import cv2
img = cv2.imread('data/45.jpg')

In [3]:
img = cv2.resize(img, INPUT_SHAPE)
img = np.expand_dims(img, 0).astype(np.float32) # add batch dim
img = img.transpose(0, 3, 1, 2) # NHWC -> NCHW
img_tokens = tokenizer_session.run(None, {'img': img})[0].reshape(1, -1)

In [4]:
decoders = decoder_session.run(None, {'encoding_indices': [img_tokens.reshape(10, 16)]})

In [7]:
decoders.shape

AttributeError: 'list' object has no attribute 'shape'

In [5]:
def tensor_to_image(image):
    #image = tensor.cpu().clone()
    image = image.squeeze(0)  # Remove the batch dimension
    image = np.transpose(image, axes = [1, 2, 0])  # Change dimensions to (H, W, C)
    image = image * 0.5 + 0.5  # Denormalize
    image = np.clip(image, 0, 1)  # Ensure pixel values are between 0 and 1
    image = (image * 255).astype(np.uint8)  # Convert to uint8
    return image
im = tensor_to_image(np.array(decoders[0]))

In [6]:
cv2.imwrite('tt.jpg', im)
cv2.waitKey(0)

-1

In [12]:
cv2.imshow("TT", im)
cv2.waitKey(0)

: 

In [12]:
type(img_tokens)

numpy.ndarray

In [15]:
b = np.array([ 726,  541,  306,  117,  759,  304,  185,  117,  921,  519,  803,  185,
          658,  666,  109,  437,  764,  928,  578,  580,  580,  219,  467,  283,
          580,  519,  467,  578,  823,  579,  928,  437,  738,  185,  467,  580,
          580,  921,  977,  117,  580,  185,  789,  580,  119,  899,  437,  726,
          738,   13, 1023,  580,  921,  921,  580,  580,  921, 1023,   12,  575,
          666,  306,   56,   56,  726,  575,  712,  489, 1023,  464,  680,  580,
          107,  184,  328,  658,  437,  712,  437,  726,  437,  437,  764,   26,
          354,  237,  354,  611,  107,  580,  666,  580,  200,   12,  658,  726,
          437,  437,  150,  899,  578,  136,  578,  185,  585,  580,  193,  112,
          219,  872,  585,  726,  726,  899,  185,  578,  580,  193,  580, 1023,
          119,  119,  921,  921,  921,  921,  865,  726,  738,  437,  119,  304,
          580,  283,  580,  611,  107,  580,  921,  921,  921,  932,  726,  437,
          738,  726,  437,  899, 1012,   28,  921,  921,  921,  921,  921,  580,
          656,  726,  437,  437])

In [16]:
a == b


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,